# 02807 Computational Tools for Data Science Group Project

## Importing the data

In [1]:
import pandas as pd
import glob
import os

In [2]:
# Read CSV files from 'data' directory
csv_files = glob.glob(os.path.join("data", "*.csv"))

In [3]:
# Generate Pandas DataFrames from CSV files
df_all = [pd.read_csv(file) for file in csv_files]

In [4]:
# Concatenate DataFrames
df = pd.concat(df_all, ignore_index=True)

## Cleaning the data

In [5]:
# Inspect data
df.head()

,carrier_code,flight_number,origin_airport,destination_airport,date,scheduled_elapsed_time,tail_number,departure_delay,arrival_delay,delay_carrier,...,HourlyPrecipitation_x,HourlyStationPressure_x,HourlyVisibility_x,HourlyWindSpeed_x,STATION_y,HourlyDryBulbTemperature_y,HourlyPrecipitation_y,HourlyStationPressure_y,HourlyVisibility_y,HourlyWindSpeed_y
0,AS,121,SEA,ANC,2019-07-01,215,N611AS,0,-16,0,...,0.0,29.60,10.0,6.0,7.027253e+10,57.0,0.0,30.04,10.0,3.0
1,B6,1092,BQN,FLL,2019-07-01,155,N589JB,-19,-40,0,...,NaN,29.81,10.0,9.0,7.478301e+10,78.0,0.0,30.01,10.0,0.0
2,B6,668,PSE,MCO,2019-07-01,175,N662JB,73,69,0,...,NaN,NaN,NaN,NaN,7.220501e+10,76.0,0.0,29.93,10.0,3.0
3,F9,402,LAX,DEN,2019-07-01,143,N706FR,-1,-7,0,...,0.0,29.60,10.0,6.0,7.256500e+10,62.0,0.0,24.69,10.0,8.0
4,F9,790,PDX,DEN,2019-07-01,156,N350FR,21,10,0,...,0.0,29.92,10.0,6.0,7.256500e+10,62.0,0.0,24.70,8.0,10.0


In [6]:
# Inspect data types and missing values
pd.DataFrame({'Data Type': df.dtypes, 'Missing Values': df.isnull().sum(), 'Percentage Missing Values': round((df.isnull().sum() / len(df)) * 100, 2)})

,Data Type,Missing Values,Percentage Missing Values
carrier_code,object,0,0.00
flight_number,int64,0,0.00
origin_airport,object,0,0.00
destination_airport,object,0,0.00
date,object,0,0.00
scheduled_elapsed_time,int64,0,0.00
tail_number,object,13556,0.25
departure_delay,int64,0,0.00
arrival_delay,int64,0,0.00
delay_carrier,int64,0,0.00


### Missing values

In [7]:
# Drop 'tail_number' column (due to missing values and insignificance)
df = df.drop(['tail_number'], axis=1)

In [8]:
# Drop rows with any missing values across all columns other than the departure and arrival columns (cancelled flights)
df = df.dropna(subset=df.columns.difference(['actual_departure_dt', 'actual_arrival_dt']))

### Data types

In [9]:
# Convert date columns to datetime format
date_columns = ['date', 'scheduled_departure_dt', 'scheduled_arrival_dt', 'actual_departure_dt', 'actual_arrival_dt']

for col in date_columns:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [10]:
# Inspect adjusted data types and missing values
pd.DataFrame({'Data Type': df.dtypes, 'Missing Values': df.isnull().sum(), 'Percentage Missing Values': round((df.isnull().sum() / len(df)) * 100, 2)})

,Data Type,Missing Values,Percentage Missing Values
carrier_code,object,0,0.0
flight_number,int64,0,0.0
origin_airport,object,0,0.0
destination_airport,object,0,0.0
date,datetime64[ns],0,0.0
scheduled_elapsed_time,int64,0,0.0
departure_delay,int64,0,0.0
arrival_delay,int64,0,0.0
delay_carrier,int64,0,0.0
delay_weather,int64,0,0.0


### Feature engineering

In [11]:
import numpy as np

In [12]:
# Function to categorise delay times
def categorise_delay(delay):
    if delay <= 15:          # Minor delay
        return 0
    elif 0 < delay <= 60:    # Moderate delay
        return 1
    elif 30 < delay <= 120:  # Significant delay
        return 2
    else:                    # Severe delay
        return 3

In [13]:
# Add categorised delay time features to data
df['departure_delay_category'] = df['departure_delay'].apply(categorise_delay)
df['arrival_delay_category'] = df['arrival_delay'].apply(categorise_delay)

In [14]:
# Inspect updated data
df.head()

,carrier_code,flight_number,origin_airport,destination_airport,date,scheduled_elapsed_time,departure_delay,arrival_delay,delay_carrier,delay_weather,...,HourlyVisibility_x,HourlyWindSpeed_x,STATION_y,HourlyDryBulbTemperature_y,HourlyPrecipitation_y,HourlyStationPressure_y,HourlyVisibility_y,HourlyWindSpeed_y,departure_delay_category,arrival_delay_category
0,AS,121,SEA,ANC,2019-07-01,215,0,-16,0,0,...,10.0,6.0,7.027253e+10,57.0,0.0,30.04,10.0,3.0,0,0
3,F9,402,LAX,DEN,2019-07-01,143,-1,-7,0,0,...,10.0,6.0,7.256500e+10,62.0,0.0,24.69,10.0,8.0,0,0
4,F9,790,PDX,DEN,2019-07-01,156,21,10,0,0,...,10.0,6.0,7.256500e+10,62.0,0.0,24.70,8.0,10.0,1,0
5,F9,662,SFO,DEN,2019-07-01,158,106,84,0,0,...,10.0,13.0,7.256500e+10,62.0,0.0,24.70,8.0,10.0,2,2
6,AS,135,ORD,ANC,2019-07-01,400,-9,-4,0,0,...,10.0,7.0,7.027253e+10,57.0,0.0,30.06,10.0,0.0,0,0


In [15]:

# Function to categorise weather conditions relative to historical weather conditions AT LOCATION
def categorise_weather(df, weather_col, station_col):
    station_stats = df.groupby(station_col)[weather_col].agg(['mean', 'std']).reset_index()
    
    df = df.merge(station_stats, on=station_col, how='left', suffixes=('', '_stats'))
    
    conditions = [
        (df[weather_col] < df['mean'] - df['std']),  # Much Lower than average
        (df[weather_col] < df['mean']),              # Lower than average
        (df[weather_col] < df['mean'] + df['std']),  # Higher than average
        (df[weather_col] >= df['mean'] + df['std'])  # Much Higher than average
    ]
    
    categories = [-2, -1, 1, 2]

    df[f'{weather_col}_category'] = np.select(conditions, categories, default='Average')
    
    df.drop(['mean', 'std'], axis=1, inplace=True)
    
    return df

In [16]:
# Add categorised weather condition features to data
station_columns = ['STATION_x', 'STATION_y']
weather_columns = ['HourlyDryBulbTemperature_x', 'HourlyPrecipitation_x', 'HourlyStationPressure_x', 'HourlyVisibility_x', 'HourlyWindSpeed_x', 'HourlyDryBulbTemperature_y', 'HourlyPrecipitation_y', 'HourlyStationPressure_y', 'HourlyVisibility_y', 'HourlyWindSpeed_y']

for station_col in station_columns:
    for weather_col in weather_columns:
        df = categorise_weather(df, weather_col, station_col)

In [17]:
# Inspect updated data
df.head()

,carrier_code,flight_number,origin_airport,destination_airport,date,scheduled_elapsed_time,departure_delay,arrival_delay,delay_carrier,delay_weather,...,HourlyDryBulbTemperature_x_category,HourlyPrecipitation_x_category,HourlyStationPressure_x_category,HourlyVisibility_x_category,HourlyWindSpeed_x_category,HourlyDryBulbTemperature_y_category,HourlyPrecipitation_y_category,HourlyStationPressure_y_category,HourlyVisibility_y_category,HourlyWindSpeed_y_category
0,AS,121,SEA,ANC,2019-07-01,215,0,-16,0,0,...,1,-1,1,1,-1,1,-1,2,1,-1
1,F9,402,LAX,DEN,2019-07-01,143,-1,-7,0,0,...,-1,-1,1,1,-1,1,-1,1,1,-1
2,F9,790,PDX,DEN,2019-07-01,156,21,10,0,0,...,-1,-1,1,1,-1,1,-1,1,-1,1
3,F9,662,SFO,DEN,2019-07-01,158,106,84,0,0,...,-1,-1,1,1,1,1,-1,1,-1,1
4,AS,135,ORD,ANC,2019-07-01,400,-9,-4,0,0,...,1,-1,-1,1,-1,1,-1,2,1,-2
